In [1]:
from robobrowser import RoboBrowser
import plotly.plotly as py
import pandas as pd
import networkx as nx
from cachier import cachier
import geocoder

browser = RoboBrowser(history=True, parser='html5lib')
base_url = 'https://flightaware.com/live/fleet/'
query = 'JAL' ## 
airlines = "LAN", "JAL", "BAW", "AAL", "CCA", 'AFR'
df = pd.DataFrame()

for query in airlines:
    print(query)
    for i in range(2):
        url = base_url + query + "?;offset=" + str(i*20)
        browser.open(url)
        table=browser.select('.prettyTable')[0]
        t=table.__str__()
        if 'Sorry' in t:
                break
        tdf=pd.read_html(t, header=1)[0]
        df = df.append(tdf)
        print(tdf.head(3))

LAN
    Ident  Type                                             Origin  \
0  LAN159  A321                                El Loa (CJC / SCCF)   
1  LAN174  A320  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)   
2  LAN502  B789  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)   

                                         Destination        Departure  \
0  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)  Sun 09:45PM -03   
1                   Diego Aracena Int'l (IQQ / SCDA)  Sun 09:09PM -03   
2                                  Miami Intl (KMIA)  Sun 10:45PM -03   

  EstimatedArrival Time  
0       Sun 11:27PM -03  
1       Sun 11:05PM -03  
2       Mon 06:15AM EDT  
JAL
    Ident  Type                             Origin  \
0    JAL1  B77W          San Francisco Intl (KSFO)   
1   JAL11  B788      Dallas-Fort Worth Intl (KDFW)   
2  JAL113  B772  Tokyo Int'l (Haneda) (HND / RJTT)   

                         Destination        Departure EstimatedArrival Time  
0  Tokyo Int'l (Han

In [2]:
G = nx.from_pandas_dataframe(df, 'Origin', 'Destination')
centrality=nx.betweenness_centrality(G)

In [7]:
from joblib import Memory
memory = Memory(cachedir='tmp', verbose=0)

@memory.cache
def get_latlng(query):
    g=geocoder.google(query)
    if g:
        return g.latlng
    else:
        return None, None
    
great_lines = []
airports = []
airport_data = []
seen=[]
great_lines_2=[]
airports = pd.DataFrame(columns=['label','lat','lon','size'])

    
i=0
for index,row in df.iterrows():
    origin = row.Origin
    destination = row.Destination
    olat, olon = get_latlng(origin)
    dlat, dlon = get_latlng(destination)
    if olat and dlat:
        if origin not in seen:
            seen.append(origin)
            print(origin, olat, olon)
        if destination not in seen:
            seen.append(destination)
            print(destination, dlat, dlon)
        great_lines.append(dict(
        type = 'scattergeo',
        lat = [ olat, dlat ],
        lon = [ olon, dlon ],
        hoverinfo = 'Ident',
        mode = 'lines',
        line = dict(
            width = 1,
            color = 'rgba(255,0,0,0.5)',
            ),
            )
        )
        airports.loc[i]=[origin, olat, olon, 0.0]
        airports.loc[i+1]= [destination, dlat, dlon, 0.0]
        i+=2
        
for key, item in centrality.items():
    airports.loc[airports.label==key, 'size'] = (item*40)+10
    
airport_markers = [ dict(
        type = 'scattergeo',
        lon = airports['lon'],
        lat = airports['lat'],
        hoverinfo = 'label',
        text = airports['label'],
        mode = 'markers',
        marker = dict( 
            size=airports['size'], 
            color='red',   
        ))]
    
layout = dict(
        title = 'Great lines over globe<br>(Click and drag to rotate)',
        width = 800,
        height = 800,
        showlegend = False,         
        geo = dict(
            projection = dict( 
                type = 'orthographic',          
            ),
            
        )
    )
    
fig = dict( data=great_lines+airport_markers, layout=layout )
py.iplot( fig, validate=False, filename='d3-globe' )

El Loa (CJC / SCCF) -22.4989612 -68.90696609999999
Comodoro Arturo Merino Benitez Int'l (SCL / SCEL) -33.3911894 -70.79723539999999
Diego Aracena Int'l (IQQ / SCDA) -20.5364084 -70.1805181
Miami Intl (KMIA) 25.795865 -80.2870457
Jorge Chavez Int'l (LIM / SPJC) -12.0240527 -77.112036
John F Kennedy Intl (KJFK) 40.6413111 -73.77813909999999
El Dorado Int'l (BOG / SKBO) 4.7014128 -74.1444969
El Tepual Int'l (PMC / SCTE) -41.433513 -73.0987251
Lic. Benito Juarez Int'l (MEX / MMMX) 19.4360762 -99.07190829999999
São Paulo-Guarulhos Int'l (GRU / SBGR) -23.4345529 -46.478126
Barajas Int'l (MAD / LEMD) 40.4839361 -3.5679515
Sydney (SYD / YSSY) -33.9399228 151.1752764
Auckland (AKL / NZAA) -37.0082476 174.7850358
Dallas-Fort Worth Intl (KDFW) 32.8998091 -97.0403352
Narita Int'l (NRT / RJAA) 35.7719867 140.3928501
Tokyo Int'l (Haneda) (HND / RJTT) 35.5493932 139.7798386
Osaka Int'l (Itami) (ITM / RJOO) 34.7895799 135.4380895
Aomori (AOJ / RJSA) 40.7382794 140.6890971
Vancouver Int'l (CYVR) 49.196

In [ ]:
from nxpd import draw
from nxpd import nxpdParams
nxpdParams['show'] = 'ipynb'
G.graph['dpi'] = 120
draw(G)